In [1]:
import requests
import urllib.parse as urlparse
import geopy
from geopy import distance

In [2]:
pip install  geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
class Route:
    def __init__(self, headers={"X-NCP-APIGW-API-KEY-ID": "your_API_KEY_ID",
           "X-NCP-APIGW-API-KEY": "your_API_KEY"}):
        self.headers = headers
    
    # 각 출발지 -> 위경도 변환
    def addr_to_xy(self):
        # 주소값 입력
        self.d1_name = input("출발지 1을 입력하세요. : ")
        self.d2_name = input("출발지 2를 입력하세요. : ")
        # URL 설정
        self.d1_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d1_name)
        self.d2_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d2_name)
        # 리퀘스트
        self.response1 = requests.get(self.d1_url, headers=self.headers)
        self.response2 = requests.get(self.d2_url, headers=self.headers)
        # JSON 파싱하여 위경도 추출
        self.d1_x = self.response1.json()["addresses"][0]["x"] 
        self.d1_y = self.response1.json()["addresses"][0]["y"]
        self.d2_x = self.response2.json()["addresses"][0]["x"]
        self.d2_y =  self.response2.json()["addresses"][0]["y"]
        return self.d1_x, self.d1_y, self.d2_x, self.d2_y
    
    # 출발지간 위경도 -> 경로값 변환
    def road_path(self):
        self.d1_xy = str(self.d1_x) + "," + str(self.d1_y)
        self.d2_xy = str(self.d2_x) + "," + str(self.d2_y)
        self.d_option = "traoptimal"
        self.d_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={}&goal={}&option={}".format(self.d1_xy, self.d2_xy, self.d_option)
        self.response = requests.get(self.d_url, headers=self.headers)
        self.path = self.response.json()["route"]["traoptimal"][0]["path"]
        self.avg_path = self.response.json()["route"]["traoptimal"][0]["summary"]["distance"] / len(self.path)
        print("거리 : {}m, ".format(self.response.json()["route"]["traoptimal"][0]["summary"]["distance"]), len(self.path), ", ", self.avg_path)
        return self.path
    
    # 출발지간 대중교통 경로
    def trans_path(self, path_type=0):
        self.path_type = path_type
        self.key = 'your_key'
        self.url = 'https://api.odsay.com/v1/api/searchPubTransPathT'
        self.params = {'apiKey' : self.key,
                  'SX' : self.d1_x,
                  'SY' : self.d1_y,
                  'EX' : self.d2_x,
                  'EY' : self.d2_y,
                  'SearchPathType' : self.path_type,
                  'OPT':0,
                 }
        self.url = self.url + '?' + urlparse.urlencode(self.params)
        self.response = requests.get(self.url)
        # 경로내 정류장 위경도 추출
        self.transit_count = self.response.json()['result']['path'][0]['info']['busTransitCount'] + self.response.json()['result']['path'][0]['info']['subwayTransitCount']
        self.distance = round(self.response.json()['result']['path'][0]['info']['totalDistance'] / 1000,2)
        self.stop_lat_lng = []
        for i in range(1, 2*self.transit_count, 2):
            self.stop_info = self.response.json()['result']['path'][0]['subPath'][i]['passStopList']['stations']
            self.stop_lat_lng += [(float(a['y']),float(a['x'])) for a in self.stop_info]
        # 경로가 1km이하거나, 정류장이 5개 미만이면 모든 위경도 표출하고, 이외에는 중간 6개의 정류장 위경도만 표출
        if self.distance <= 1 or len(self.stop_lat_lng) <= 5:
            self.meet_point = self.stop_lat_lng
        else:
            self.meet_point = self.stop_lat_lng[len(self.stop_lat_lng)//2-3:len(self.stop_lat_lng)//2+3]
        print('거리: {}km'.format(self.distance))
        return self.meet_point, self.stop_lat_lng

In [4]:
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>우리 지금 만나, 당장 만나!</title>
    
</head>
<body>
<div id="map" style="width:100%;height:100vh;"></div>
<script type="text/javascript" src="https://dapi.kakao.com/v2/maps/sdk.js?appkey=your_appkey"></script>
<script>
var mapContainer = document.getElementById('map'),  
    mapOption = { 
        center: new kakao.maps.LatLng(37.5657, 126.9769), 
        level: 4 
    };
var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다
var positions = [
    {
        content: '<div>시청역</div>', 
        latlng: new kakao.maps.LatLng(37.5657, 126.9769)
    },
    {
        content: '<div>덕수궁</div>', 
        latlng: new kakao.maps.LatLng(37.5658, 126.9751)
    },
    {
        content: '<div>정동길</div>', 
        latlng: new kakao.maps.LatLng(37.5669, 126.9708)
    },
    {
        content: '<div>경찰박물관</div>',
        latlng: new kakao.maps.LatLng(37.5690, 126.9697)
    }
];

for (var i = 0; i < positions.length; i ++) {
    var marker = new kakao.maps.Marker({
        map: map, 
        position: positions[i].latlng
    });

    var infowindow = new kakao.maps.InfoWindow({
        content: positions[i].content 
    });
    kakao.maps.event.addListener(marker, 'mouseover', makeOverListener(map, marker, infowindow));
    kakao.maps.event.addListener(marker, 'mouseout', makeOutListener(infowindow));
}
function makeOverListener(map, marker, infowindow) {
    return function() {
        infowindow.open(map, marker);
    };
}
function makeOutListener(infowindow) {
    return function() {
        infowindow.close();
    };
}
</script>
</body>
</html>

SyntaxError: invalid syntax (Temp/ipykernel_2652/1919312645.py, line 1)

In [5]:
import requests, json

def send_msg(webhook_url, mag, channel="#dss", username="맛집찾아봇"):
    payload = {"channel":channel, "username":username, "text":mag}
    requests.post(webhook_url, data = json.dumps(payload))

In [6]:
from flask import *
import libs.roadmatzip as roadmatzip
import libs.slack as slack
import time

app = Flask(__name__)
naver_id= "your_id"
naver_secret = "your_secret_key"
odsay_key = "your_key"
webhook_url = 'your_url'

@app.route("/")
def index():
    return "server is running!"

@app.route("/bot", methods=["POST"])
def bot():
    username = request.form.get("user_name")
    token = request.form.get("token")
    text = request.form.get('text')
    print(username, token, text)
    text = text.replace("matzip! ", "")
    if len(text.split("/")) != 4:
        slack.send_msg(webhook_url, "'주소1/주소2/교통수단/음식카테고리' 형식으로 입력해주세요.")
        slack.send_msg(webhook_url, "(교통수단:자동차, 대중교통 /음식카테고리:한식, 양식, 디저트, 일식, 바, 중식, 분식, 기타, 동남아식, 뷔페)")
        return Response(), 200
    
    add1, add2, by, category = text.split("/")[0], text.split("/")[1], text.split("/")[2], text.split("/")[3]
    
    if by == "자동차":
        msg = roadmatzip.addr_to_xy(roadmatzip.road_path, add1, add2, category)
        slack.send_msg(webhook_url, "{}경로 내 {} 맛집 당장만나별점 top5".format(by, category))
        time.sleep(1)
        for i in range(5):
            slack.send_msg(webhook_url, "{}위: {}, {}, {}".format((i+1), msg.iloc[i, 0], msg.iloc[i, 6], msg.iloc[i, 2]))
            time.sleep(1)
    elif by == "대중교통":
        msg = roadmatzip.addr_to_xy(roadmatzip.trans_path, add1, add2, category)
        slack.send_msg(webhook_url, "{}경로 내 {} 맛집 당장만나별점 top5".format(by, category))
        time.sleep(1)
        for i in range(5):
            slack.send_msg(webhook_url, "{}위: {}, {}, {}".format((i+1), msg.iloc[i, 0], msg.iloc[i, 6], msg.iloc[i, 2]))
            time.sleep(1)

    return Response(), 200
app.run(debug = True)

ModuleNotFoundError: No module named 'libs.roadmatzip'